In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
import glob
import re

In [2]:
def getProducts(i_list, p_data):
    """
    Return a list of the products that contain at least one ingredient
    in the given list.
    :param i_list: List of ingredients to search for
    :param p_data: Product data to search in
    :return: List of products that contain at least one ingredient in i_list
    """
    i_set = set(i_list)
    out = []
    for k, v in p_data.items():
        i = 0
        found = False
        while i < len(v['ingredients']) and not found:
            found = v['ingredients'][i] in i_set
            i += 1
        if found:
            out.append(k)

    return out


In [3]:
with open("finaldata.json", "r") as f:
    data = json.loads("\n".join(f.readlines()))

with open("ingredients.json", "r", errors='ignore') as f:
    u_ingredients = json.loads("\n".join(f.readlines()))

with open("concerns.json", "r") as f:
    u_concerns = json.loads("\n".join(f.readlines()))
    
with open("relevant_types.json", "r") as f:
    relevant_product_types = json.loads("\n".join(f.readlines()))

In [4]:
for _, v in data.items():
    i_string = v['ingredients']
    if i_string[-1] == '.': i_string = i_string[:len(i_string) - 1]
    i_list = re.split(", |\. ", i_string)
    v['ingredients'] = [(re.sub(r".*:", "", s)).strip() for s in i_list]

In [5]:
categories = {}

for k, v in u_concerns.items():
    d = {
        "concerns": ", ".join(v),
        "products": getProducts(u_ingredients[k], data)
    }
    categories[k.lower()] = d

# Construct auxiliary data structures
num_products = len(data)
category_to_index = {name: index for index, name in enumerate(categories)}
products_to_indices = {k: v for k, v in zip(data.keys(), range(num_products))}
indices_to_products = {v: k for k, v in products_to_indices.items()}

In [6]:
def create_product_types_dict():
    """Returns a dictionary to map product types to Boolean arrays 
    (True if index corresponds to a product of that type).
    
    Params: {}
    Returns: (String -> Numpy Array) Dict 
    """
    product_types = {}
    product_files = glob.glob("./product_types_lists/*.json")
    if len(product_files) == 0: 
        print('can\'t read')  
    
    for p_file in product_files:
        name = p_file[22:-5]
        with open(p_file) as json_file:
            data = json.load(json_file)
        p_arr = np.full(num_products, False)
        for p in data:
            if p not in products_to_indices.keys():
                continue
            p_arr[products_to_indices[p]] = True
        product_types[name] = p_arr
    
    return product_types

In [7]:
def create_price_ranges():
    """Returns a dictionary to map price ranges to Boolean arrays 
    (True if index corresponds to a product within the budget).
    
    Params: {}
    Returns: (String -> Numpy Array) Dict 
    """
    price_ranges = dict([('under $15', np.full(num_products, False)), ('$15-30', np.full(num_products, False)),
                         ('$30-50', np.full(num_products, False)), ('$50-75', np.full(num_products, False)), 
                         ('$75+', np.full(num_products, False))])
    for k,v in data.items():
        if v['price'] < 15:
            price_ranges['under $15'][products_to_indices[k]] = True
        elif v['price'] < 30:
            price_ranges['$15-30'][products_to_indices[k]] = True
        elif v['price'] < 30:
            price_ranges['$30-50'][products_to_indices[k]] = True
        elif v['price'] < 30:
            price_ranges['$50-75'][products_to_indices[k]] = True
        else :
            price_ranges['$75+'][products_to_indices[k]] = True
    
    return price_ranges

In [8]:
def create_claims():
    """Returns a dictionary to map price ranges to Boolean arrays 
    (True if index corresponds to a product within the budget).
    
    Params: {}
    Returns: (String -> Numpy Array) Dict 
    """
    price_ranges = dict([('under $15', np.full(num_products, False)), ('$15-30', np.full(num_products, False)),
                         ('$30-50', np.full(num_products, False)), ('$50-75', np.full(num_products, False)), 
                         ('$75+', np.full(num_products, False))])
    for k,v in data.items():
        if v['price'] < 15:
            price_ranges['under $15'][products_to_indices[k]] = True
        elif v['price'] < 30:
            price_ranges['$15-30'][products_to_indices[k]] = True
        elif v['price'] < 30:
            price_ranges['$30-50'][products_to_indices[k]] = True
        elif v['price'] < 30:
            price_ranges['$50-75'][products_to_indices[k]] = True
        else :
            price_ranges['$75+'][products_to_indices[k]] = True
    
    return price_ranges

In [9]:
product_types = create_product_types_dict()

In [10]:
price_ranges = create_price_ranges()

In [11]:
def adjust_sensitivity(ranking, sensitive):
    """Returns the ranking after adjusting scores based on skin sensiivity.
    
    Params: {ranking: Numpy Array,
             sensitive: Boolean}
    Returns: Numpy Array 
    """
    if sensitive:
        for prod in categories['abrasive/scrub']['products']:
            ranking[products_to_indices[prod]] *= 0.5
        for prod in categories['perfuming']['products']:
            ranking[products_to_indices[prod]] *= 0.5
            
        for prod in categories['soothing']['products']:
            ranking[products_to_indices[prod]] *= 1.5
    return ranking

In [12]:
def adjust_skin_type(ranking, s_type):
    """Returns the ranking after adjusting scores based on skin type.
    
    Params: {ranking: Numpy Array,
             s_type: String}
    Returns: Numpy Array 
    """
    if s_type == 'oily':
        ranking[product_types['face_oil_products']] *= 0.5
            
        for prod in categories['absorbent/mattifier']['products']:
            ranking[products_to_indices[prod]] *= 1.5
        ranking[product_types['bha_products']] *= 1.5
        ranking[product_types['oil_absorbing_products']] *= 1.5
    
    elif s_type == 'dry':
        for prod in categories['absorbent/mattifier']['products']:
            ranking[products_to_indices[prod]] *= 0.5
        ranking[product_types['oil_absorbing_products']] *= 0.5
            
        for prod in categories['soothing']['products']:
            ranking[products_to_indices[prod]] *= 1.5
    
    elif s_type == 'combo':
        pass
    
    return ranking

In [13]:
def cos_sim(c, tfidf_mat, category_to_idx):
    """Returns the cosine similarity of the query and a concern list.
    
    Params: {c: String,
             tfidf_mat: np.ndarray,
             category_to_idx: Dict}
    Returns: Float 
    """
    # query is last row
    v1 = tfidf_mat[len(tfidf_mat)-1]
    v2 = tfidf_mat[category_to_idx[c]]
    num = np.dot(v1, v2)
    
    denom = max((np.linalg.norm(v1)*np.linalg.norm(v2)), 1e-7)
    return num/denom

In [14]:
def claims_similarity(query, product_info, prod_to_idx):
    """ Finds cosine similarity between input query (concerns) and each product's claims. 
        Returns a numpy array with each product's score.
        
        Params: {query: (user input) String,
                 product_info: (product -> Dict) Dict,
                 prod_to_idx: (product -> index) Dict}
        Returns: Numpy Array
    """
    result = np.zeros(len(prod_to_idx))
                      
    tfidf_vec = TfidfVectorizer(stop_words = 'english')
    lst = [product_info[k]['claims'] for k in product_info.keys()]
    lst.append(query)
    tfidf_mat = tfidf_vec.fit_transform(lst).toarray()
    
    for k,v in product_info.items():
        sim = cos_sim(k, tfidf_mat, prod_to_idx)
        #print(k, sim)
        result[prod_to_idx[k]] += sim
        
    return result

In [15]:
def concern_similarity(query, category_info, prod_to_idx, category_to_idx):
    """ Finds cosine similarity between input query (concerns) and each product category's concern list. 
        Returns a numpy array with each product's score, based on the categories they are in.
        
        Params: {query: (user input) String,
                 category_info: (category -> Dict) Dict,
                 prod_to_idx: (product -> index) Dict, 
                 category_to_idx: (category -> index) Dict}
        Returns: Numpy Array
    """
    result = np.zeros(len(prod_to_idx))
                      
    tfidf_vec = TfidfVectorizer(stop_words = 'english')
    lst = [category_info[k]['concerns'] for k in categories.keys()]
    lst.append(query)
    tfidf_mat = tfidf_vec.fit_transform(lst).toarray()
    
    for k,v in category_info.items():
        sim = cos_sim(k, tfidf_mat, category_to_idx)
        for p in v['products']:
            result[prod_to_idx[p]] += sim
            
        # added adjustments
        for category in relevant_product_types[k]['relevant']:
            result[product_types[category]] *= 1.5
        for category in relevant_product_types[k]['irrelevant']:
            result[product_types[category]] *= 0.1
        
    return result

In [16]:
def rank_products(query, category_info, prod_to_idx, idx_to_prod, product_info, category_to_idx,
                 product_types, price_ranges, product_type=None, skin_type=None, budget=None, sensitivity=None):
    """ Returns a ranked list of products, with the most relevant at index 0.
        
        Params: {query: (user input) String,
                 category_info: (category -> Dict) Dict,
                 prod_to_idx: (product -> index) Dict,
                 idx_to_prod: (index -> product) Dict
                 product_info: (product -> Dict) Dict
        Returns: List
    """
    scores = concern_similarity(query, category_info, prod_to_idx, category_to_idx)
    scores += 2 * claims_similarity(query, product_info, prod_to_idx)
    if sum(scores) == 0: return 'invalid query'
    
    # ranking adjustments
    if skin_type != None:
        scores = adjust_skin_type(scores, skin_type)
    if sensitivity != None:
        scores = adjust_sensitivity(scores, sensitivity)
    
    # strict filters
    if budget != None:
        scores[np.invert(price_ranges[budget])] = 0     
    if product_type != None:
        scores[np.invert(product_types[product_type])] = 0
    
    scores_idx = [(val,prod) for prod, val in enumerate(scores)]
    rank_idx = sorted(scores_idx, key = lambda x: (x[0], product_info[idx_to_prod[x[1]]]["num faves"], 
                                            product_info[idx_to_prod[x[1]]]["price"]), reverse = True)
    for r in rank_idx:
        print(idx_to_prod[r[1]], r[0])
        print(data[idx_to_prod[r[1]]]['price'])
    ranking = list(map(lambda x: idx_to_prod[x[1]], rank_idx))
    return ranking

In [17]:
a = np.array([1,2,3])
b = np.array([True, False, False])
print(a[b])

print(products_to_indices["Benzoyl Peroxide 10%"])
print(indices_to_products[3983])
print(product_types['acne_products'][3983])

[1]
3983
Benzoyl Peroxide 10%
True


In [22]:
concerns = 'wrinkly dull skin'
ranking = rank_products(concerns, categories, products_to_indices, indices_to_products, data, category_to_index, product_types, price_ranges, product_type=None, budget=None)

Retinol Correxion Deep Wrinkle Daily Moisturizer SPF 30 3.3320583067773666
24.99
Transform Plus Goodnight Glow Retin-ALT Sleeping Crème 2.5685989878392297
55.0
Mattifying Moisturizer: Oil Control 2.450399229691081
28.0
Advanced Retinol Intensive Night Moisturizer 2.127346528570728
79.0
Bye Bye Makeup 3-in-1 Makeup Melting Cleansing Balm 1.969160302823292
38.0
Men’s Essentials Five-in-One Face Lotion Broad Spectrum SPF 15 1.813193225137447
9.49
Masks Vitamin C + AHA Resurfacing Peel 1.7984352409622875
26.99
C.E.O. Glow Vitamin C + Turmeric Face Oil 1.788240768669423
80.0
Refresh AHA BHA Vitamin C Daily Toner 1.7864991324764121
28.0
Pure Truth Melting Cleanser 1.6483776367976368
34.0
GENIUS Ultimate Anti-Aging Melting Cleanser 1.6356704851883876
38.0
Rapid Wrinkle Repair Regenerating Cream 1.5283055023180065
27.99
RetAsphere Micro Peel Retinol Glycolic Treatment 1.5239777216878805
90.0
Retinol Correxion Max Daily Hydration Creme 1.5195631759889587
22.99
C15 Super Booster 1.47385431378833

630.0
TNS Ceramide Treatment Cream 0.8456695253458841
64.0
Essential Power Cream 0.845290250340582
205.0
Vitamin C Energizing Face Spritz 0.8451707229831656
18.0
Hypoallergenic Nourishing Moisturizer 0.845041801900535
75.0
Ultra-Lite Anti-Aging Cellular Repair 0.8448640398088793
52.0
Timewise Repair Day Cream Sunscreen Broad Spectrum SPF 30-Volu-Firm 0.8448436414561424
52.0
Rejuvenating Lift for Neck and Decollete 0.8447044138116653
55.0
Regenerist Luminous Tone Perfecting Cream Moisturizer 0.8445578915101173
30.99
Cicapair Tiger Grass Color Correcting Treatment SPF 30 0.844547149598493
52.0
Re-Nutriv Replenishing Comfort Creme 0.8445055121518582
145.0
Revitalizing Night Repair Cream 0.8444278742588749
12.0
Clear Genius Clarifying Toner + Serum Clarity 5 Complex 0.8443643940451828
18.0
Rapid Tone Repair Moisturizer with Sunscreen Broad Spectrum SPF 30 0.8443076065466998
20.99
FC5 Nurturing Day Lotion with SPF 20 0.8441688641297826
40.0
Day Complex with SPF 20 0.8440841745078244
34.0
Re

Fill & Freeze Wrinkle Treatment 0.81937251925701
55.0
Pro-X Wrinkle Smoothing Cream 0.81937251925701
29.99
Hydro-Dynamic Ultimate Moisture 0.81937251925701
65.0
Rapid Wrinkle Repair Moisturizer SPF 30 0.81937251925701
20.99
Firming Complex 0.81937251925701
124.0
Pro-X Hydra Firming Cream 0.81937251925701
29.99
RE9 Advanced Restorative Day Creme SPF 20 0.81937251925701
50.0
bareVitamins Eye Rev-er Upper 0.81937251925701
21.0
Daily Moisture Lightweight Pore-Minimizing Moisturizer, for Normal or Oily Combination Skin 0.81937251925701
60.0
FIRMx Firming Night Moisturizer 0.81937251925701
125.0
miracle worker overnight 0.81937251925701
68.0
The Youth As We Know It 0.81937251925701
74.0
Wrinkle Revenge Rescue & Protect Facial Cream 0.81937251925701
54.0
Renewed Radiance Nutrifying Night Cream 0.81937251925701
16.99
Signs Up-Lifter 0.81937251925701
265.0
Eternalixir Skin-Volumizing Oil Serum 0.81937251925701
55.0
Recover-E 0.81937251925701
44.5
Eterna 27 Moisture Cream with Progenitin 0.81937

Changing Skin Night Creme 0.7362168016671622
46.0
Smart Combination Smoothing Lightweight Emulsion 0.7361738846808836
35.0
Pure Perfection 0.7359923453028295
50.0
Healing Ointment 0.735974588662215
10.99
Wrinkle Expert 45+ Moisturizer 0.7359322587905041
9.99
Nighttime Recovery Creme 0.7355429917580818
46.0
Anti-Aging Stress Cream 0.7353998087148289
220.0
Dramatic Firming Cream 0.7352578584149047
8.5
Environmental Defense Sunscreen SPF 50+ 0.7352452431864859
45.0
Super Restorative Day Cream 0.7351374053116273
122.0
Extra-Firming Day Wrinkle Lifting Cream, Special for Dry Skin 0.7351255622434536
84.0
Moisture Rich Facial Treatment 0.7349280867903933
22.0
Deep Tissue Repair Cream With Peptide K8 0.7349218851103934
150.0
Vitamin A & E Moisturizer 0.73470455375187
11.95
Pro C Illuminating Cell Defense Pads 0.73459548069612
49.5
Firming Cream, Intense 0.7345751665706691
72.0
BC Spa Resurface Multi-Acid Resurfacing Peel 0.734557513139862
50.0
Cellular Anti-Wrinkle Sun Cream SPF 30 0.734426454

52.0
BC Spa Resurface Daily Resurfacing Serum 0.6224320147015056
40.0
Age Defying Multi-Vitamin Moisturizer 0.6218084557314904
7.49
KateCeuticals Replenishing Toner 0.6202292327274106
58.0
Sulfate Free Rapid Exfoliating CleanserAM 0.6201012524194369
18.0
Eczema Soothing Creamy Oil 0.6200814678173449
15.99
Advanced Marine Biology Tonic 0.6199728705050048
100.0
Regenerist Micro-Sculpting Serum 0.6198261752381253
27.99
Moisturizing Make-up Remover Wipes 0.6197942515713636
7.49
Vitamin C Facial Serum 0.6197516738905741
42.0
Wrinkle Concentrate 0.6195323084866817
38.0
Ultra Repair Hydrating Serum 0.6193696416071344
38.0
Green Tea & Green Clay Rejuvenating Mask, for Normal Skin 0.6192612417557748
9.43
Solutions Plus+ Total Radiance Thermal Cleanser 0.618491707895373
8.0
Wild Rose Face and Eye Serum 0.6183132831738262
45.0
Wrinkle Recode Line-Transforming Melting Serum 0.618286116931085
89.0
Water Boost Hydrating Gel Cream 0.618065041720889
9.99
Retinol Tonic 0.6177663763011265
15.0
Daily Reg

...

42.0
Gentle Face Exfoliator 0.031101956944308767
24.0
Aquaphor Baby Gentle Wash & Shampoo 0.030879200476618095
6.29
Bee Pollen Night Cream 0.030760535601774625
20.0
Toleraine Ultra Intense Soothing Care 0.0306658651861408
36.99
Rosaliac AR Intense Localized Redness Intensive Serum 0.030595915522717326
39.95
All Over Cover Stick 0.030578284524574915
1.0
Adaptogen Deep Moisture Cream 0.030564488524701082
58.0
SkinActive Micellar Cleansing Water, Regular 0.03047511202034196
8.99
Acne Proofing Daily Scrub 0.030472097113386117
9.99
Facial Marula Oil 0.030271048314969174
15.99
Ultra Facial Oil-Free Cleanser 0.029945429067847133
19.5
Kombucha + 11% AHA Exfoliation Power Toner 0.029905768848583925
38.0
Omega+ Complex Moisturizer 0.029869629553068697
35.0
Gel Cleanser 0.02979554563741191
25.5
Exfoliating Scrub 0.02978451365491019
21.0
Aloe Soothing Moisture Lotion SPF 15, for Sensitive Skin 0.02972807250884467
18.5
Hydraluron+ Moisture Mist 0.029703086770320346
24.99
Nanoblur 0.0296247076701388

Elements Moisturizing Cream Cleanser 0.02296412201755699
6.99
Hyalis Hydrating Serum 0.022909384971099782
64.0
Spectro Jel for Sensitive, Blemish Prone SkinSoap & Fragrance Free 0.022896563688830585
13.99
Elements Youth Restoring Anti-Wrinkle Eye Roller 0.022853256550839914
11.99
Vaseline Jelly Original 0.022806479528678855
4.27
Original Healing Soothing Repair Creme 0.022759045363024377
13.79
Deep Action Exfoliating Scrub, Oil-Free 0.02272963597056747
5.49
Enzyme Protective Cream 0.02269467503461492
20.0
Knockout Daily Exfoliating Cleanser 0.022626646044518587
24.0
Expert Anti-Blemish Toner 0.022611309402547297
4.29
EARTH SOURCED Perfectly Natural Cleansing Gel 0.022610884620756083
18.0
Face SPF 30 Unscented Matte Tint Natural BB Cream 0.022562830333699
36.0
Zero Oil Deep Pore Cleanser 0.022537557925294785
21.5
Warming Gel Moisturising Mask 0.02253558179439479
11.0
Oil Free Gel Cleanser 0.022504962511636195
25.5
Face SPF 30 Unscented Moisturizer 0.02249645074339965
32.0
PC4MEN Shave 0

15.0
Intensive Healing Cream Daily Fragrance-Free Cream 0.013587213877499466
9.99
HQplus Lightening Gel 0.013496673570295896
34.0
Aloe Mint Cooling Shower Gel 0.013425907999282873
9.95
Organic Grape Water, for All Skin Types 0.01342317165148575
18.0
Superscreen Daily Moisturizer Broad Spectrum SPF 40 0.013363398532805717
38.0
Anti-Expression Line Cream 0.013360986262290691
37.75
Normaderm Total Mat Anti-Shine Mattifier 0.0133588869385586
24.5
Foaming Cleanser 0.013344524404873676
34.0
Jelly Glow Ball Radiance-Boosting Cleanser 0.01332399841797017
16.0
Adaptogen Soothe + Hydrate Activated Mist 0.013274521720316965
42.0
Daily Moisture Fragrance-Free Dry Skin Moisturizer 0.013261885246427236
6.49
Oil-Free Moisturizer SPF 17 Broad Spectrum Sunscreen 0.01323558633926568
24.0
Stretch Mark Minimizer 0.013228596821316234
50.0
Face Treatment Oil, Lotus, for Combination Skin 0.013225143397805013
52.0
CBD Skin Transformative Treatment Milk 0.013209523109964543
49.0
Twist-Up Long Wear Gel Liner 0.

Azulene Calming Mask 0.009855356578798428
18.0
Sebium Mat Anti-Shine Fluid 0.009853211030327503
20.95
Ultra Overnight Face Lotion 0.009834233177841573
8.99
Banish Solution 0.009793436431306445
21.0
Advantage 3-in-1 Foaming Acne Wash 0.009775417679460533
8.49
Coconut Argan Oil 0.009688318749075274
12.99
Acerola 100 0.009687129978830533
22.5
Acerola Extract 0.009651162923530798
22.5
CLEAR Regular Strength Anti-Redness Exfoliating Solution with 2% Salicylic Acid 0.009643835885483239
29.5
Skin Hydrating Booster 0.009637895822440207
56.0
Duo Dual-Sided Body Cleanser 0.009617919407361213
9.99
Intensive Repair Extra-Enriched Foot Creme 0.009594073028676521
5.79
Eye Makeup Remover, Oil-Free 0.00957635931493167
1.99
MEN Extra Strength Body and Face Lotion 0.009557178085174742
3.99
purity made simple pore extractor mask 0.009512237156096942
35.0
DEFENSE Nightly Reconditioning Moisturizer 0.009465680525893392
29.0
Citrus Ginger Argan Oil 0.009439808551030018
12.99
ClarinsMen Active Face Wash 0.00

21.0
Antibacterial Facial Cleanser PHA 0.0
30.0
Coconut Milk Eye Makeup Remover 0.0
5.95
Oil Free Matifying Fluid 0.0
30.0
Blemish + Age Defense Salicylic Acid Acne Treatment 0.0
82.0
Liquid Facial Soap Oily Skin Formula 0.0
16.5
Yes to Cucumbers Soothing Facial Towelettes 0.0
5.99
Original Skin Retexturizing Mask With Rose Clay 0.0
25.0
SPF 15 Alcohol-Free Sunscreen Spray 0.0
18.99
Mederma + SPF 30 Scar Cream 0.0
34.49
SPF 30 Sun Protection Stick 0.0
12.0
Vinyl Liquid Liner 0.0
7.0
Unblemish Acne Treatment Sulfur Wash 0.0
41.0
Simply Clean Pore-Refining Gel Cleanser for Combination or Oily Skin 0.0
34.0
Positively Ageless Youth Perfecting Moisturizer SPF 30 0.0
19.99
Apricot & Manuka Honey Cleansing Stick 0.0
7.99
Pro Sculpting Duo 0.0
39.0
Sebium H2O 0.0
23.0
Everyday Sunscreen Broad Spectrum SPF 50 0.0
19.0
Hyaluronic Eye Cream 0.0
18.0
PanOxyl Acne Foaming Wash, 10% Benzoyl Peroxide 0.0
10.79
Olive & Aloe Soap 0.0
2.99
Gel Plus, 15 AHA 0.0
45.0
Ultra Rapid Action Pads 0.0
8.99
Goat

In [19]:
# abrasive/scrub: 'acne, oil, oily, pores, pore, breakout, breakouts, wrinkles, lines, liney, texture, textured, smooth, smoother, smoothness, dull, dullness, dirt, dirty, makeup, clean, cleaning, comedones, blackhead, bleackheads, whitehead, whiteheads'
# absorbent/mattifier: 'acne, oil, oily, pores, pore, pimple, pimples, breakout, breakouts, texture, textured, blackhead, bleackheads, whitehead, whiteheads'
# anti-acne: 'acne, red, redness, oil, oily, pores, pore, pimple, pimples, breakout, breakouts, texture, textured, inflamed, inflammation, irritate, irritated, irritation, bump, bumps, bumpy'
# antimicrobial/antibacterial: 'acne, oil, oily, pimple, pimples, breakout, texture, textured, bump, bumps, bumpy, comedones, blackhead, bleackheads, whitehead, whiteheads, pore, pores, dirt, dirty, makeup, clean, cleaning'
# antioxidants: 'aging, age, wrinkles, wrinkly, lines, liney, texture, textured, rough, roughness, uneven, even, repair, repaired, repairs, protect, protection, sun, damage, damaged, dead, old, dull, dullness, pigmentation, hyperpigmentation, spots, spotty, dots, dotty, loose, droopy, drooping, sag, saggy, sags, sagging, elastic, elasticity, firm, firmer, firmness fresh, fresher, freshness, bright, brighter, brightness'
# astringent: 'oil, oily, pores, pore, dirt, dirty, makeup, clean, cleaning, tone, toning, toner'
# buffering: 'irritate, irritated, irritation, inflamed, inflammation, sensitive, sensitivity'
# cell-communicating ingredient: 'aging, age, wrinkles, wrinkly, lines, liney, texture, rough, roughness, smooth, smoother, smoothness, uneven, even, damage, damaged, dead, old, dull, dullness, pigmentation, hyperpigmentation, spots, spotty, dry, dryness, loose, droopy, drooping, sag, saggy, sags, sagging, elastic, elasticity, firm, firmer, firmness, hydrate, hydrated, hydration, moisture, moisturizer, soft, softer, soften, fresh, fresher, freshness'
# chelating: '' # no concerns, just stabilizes products
# colorant: '' # no concerns, just colors products
# deodorant: 'smell, smelling, odor, sweat, sweaty, sweatiness' # unlikely to be a part of our data, but just in case
# emollient: 'itch, itchy, itchiness, redness, red, rosacea, texture, textured, rough, roughness, uneven, damage, damaged, dead, smooth, smoother, smoothness, inflamed, inflammation, irritate, irritated, irritation, dry, dryness, hydrate, hydrated, hydration, moisture, moisturizer, soft, softer, soften'
# emulsifying: '' # no concerns, just helps mix ingredients
# emulsion stabilising: '' # no concerns, just helps products stay mixed/stable
# exfoliant: 'acne, oil, oily, pores, pore, breakout, breakouts, wrinkles, lines, liney, texture, textured, smooth, smoother, smoothness, dull, dullness, dirt, dirty, makeup, clean, cleaning, comedones, blackhead, bleackheads, whitehead, whiteheads, bright, brighter, brightness, fresh, fresher, freshness'
# moisturizer/humectant: 'dry, dryness, peel, peeling, itch, itchy, itchiness, moisture, moisturizer, hydrate, hydrated, hydration, texture, rough, roughness, smooth, smoother, smoothness, soft, softer, soften, fresh, fresher, freshness'
# perfuming: '' # no concerns, just fragrance
# preservative: '' # no concerns, just for product longevity
# skin brightening: 'sun, damage, damaged, repair, repaired, repairs, dull, dullness, pigmentation, hyperpigmentation, dark, darker, brown, brownish, spots, spotty, dots, dotty, brighter, brightness, even, uneven, tone, toner, toning'
# skin-identical ingredient: 'aging, age, wrinkles, wrinkly, lines, liney, texture, rough, roughness, smooth, smoother, smoothness, uneven, even, protect, protection, damage, damaged, dead, old, dull, dullness, pigmentation, hyperpigmentation, spots, spotty, dry, dryness, loose, droopy, drooping, sag, saggy, sags, sagging, elastic, elasticity, firm, firmer, firmness, hydrate, hydrated, hydration, moisture, moisturizer, soft, softer, soften, fresh, fresher, freshness'
# solvent: '' # no concerns, just for dissolving ingredients
# soothing: 'itch, itchy, itchiness, redness, red, rosacea, soothe, soothing, smooth, smoother, smoothness, inflamed, inflammation, swelling, irritate, irritated, irritation, patches, patch, acne, breakout, breakouts, pimple, pimples, sensitive, sensitivity, gentle'
# sunscreen: 'sun, damage, damaged, repair, repaired, repairs, protect, protection, dull, dullness, pigmentation, hyperpigmentation, dark, darker, brown, brownish, spots, spotty, dots, dotty, brighter, brightness, even, uneven, tone, toner, toning, aging, age, wrinkles, wrinkly, lines, liney'
# surfactant/cleansing: 'dirt, dirty, makeup, clean, cleaning'
# viscosity controlling: '' # no concerns, just for product thickness